# Websocket Message를 종류별로 구성함.

## 타입별로 웹소켓 데이터를 수신하고 분석용도로 사용한다.
- ticker: 개별 심볼에 대한 전체 티커 정보 제공
- trade: 개별 거래 정보 제공
    - "m" : 매도자 관점 taker주문(시장가)
- miniTicker: 심볼별 간소화된 티커 정보 제공
- depth: 주문서 정보 제공
- aggTrade: 집계된 거래 정보 제공

# 웹소켓 데이터 저장 스토리지 클라스

In [1]:

import asyncio
import nest_asyncio
from pprint import pprint
from typing import List
import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))
from Workspace.Services.PublicData.Receiver.FuturesMarketWebsocket import FuturesMarketWebsocket

nest_asyncio.apply()
symbol = ["BTCUSDT"]
instance = FuturesMarketWebsocket(symbol)
ws_types = ["ticker", "trade", "miniTicker", "depth", "aggTrade"]

class storage:
    __slots__ = tuple(ws_types)
    def __init__(self):
        for attr in self.__slots__:
            setattr(self, attr, [])

    def set_data(self, ws_type:str, data:List):
        setattr(self, ws_type, data)
    
    def get_data(self, ws_type:str):
        return getattr(self, ws_type)

storage_message = storage()

In [2]:
async def main(stream_type):
    print(f"  ⏳ Websocket Stream({stream_type}) 연결 시도.")
    await instance.open_stream_connection(stream_type)
    print(f"  🔗 Websocket Stream({stream_type}) 연결 완료.")
    message = await instance.receive_message()
    print(f"  📩 message 수신")
    pprint(message)
    await instance.close_connection()
    print(f"  ⛓️‍💥 Websocket Stream({stream_type}) 연결 종료.")

async def ws_cycle(stream_type:str, count:int):
    print(f"  ⏳ Websocket Stream({stream_type}) 연결 시도.")
    await instance.open_stream_connection(stream_type)
    print(f"  🔗 Websocket Stream({stream_type}) 연결 완료.")
    print(f"  📩 message 수신 시작")
    result = []
    for _ in range(count):
        message = await instance.receive_message()
        print(type(message))
        result.append(message)
    print(f"  📩 message 수신 완료")
    await instance.close_connection()
    print(f"  ⛓️‍💥 Websocket Stream({stream_type}) 연결 종료.")
    return result

In [3]:
ws_type = ws_types[0]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(ticker) 연결 시도.
  🔗 Websocket Stream(ticker) 연결 완료.
  📩 message 수신 시작
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(ticker) 연결 종료.
[{'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385472907, 's': 'BTCUSDT', 'p': '1305.90', 'P': '1.582', 'w': '82357.76', 'c': '83846.20', 'Q': '0.047', 'o': '82540.30', 'h': '84088.20', 'l': '81113.50', 'v': '176627.893', 'q': '14546677916.42', 'O': 1742299020000, 'C': 1742385472903, 'F': 6114867777, 'L': 6117923264, 'n': 3054887}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385474903, 's': 'BTCUSDT', 'p': '1313.30', 'P': '1.591', 'w': '82357.79', 'c': '83853.60', 'Q': '0.004', 'o': '82540.30', 'h': '84088.20', 'l': '81113.50', 'v': '176630.836', 'q': '14546924687.44', 'O': 1742299020000, 'C': 1742385474899, 'F': 6114867777, 'L': 6117923329, 'n': 30549

In [4]:
ws_type = ws_types[1]
receiver_count = 20
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(trade) 연결 시도.
  🔗 Websocket Stream(trade) 연결 완료.
  📩 message 수신 시작
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(trade) 연결 종료.
[{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385491584, 'T': 1742385491584, 's': 'BTCUSDT', 't': 6117923730, 'p': '83849.70', 'q': '0.013', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385491657, 'T': 1742385491657, 's': 'BTCUSDT', 't': 6117923731, 'p': '83849.60', 'q': '0.050', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492047, 'T': 1742385492047, 's': 'BTCUSDT', 't': 6117923732, 'p': '83849.60', 'q': '0.514', 'X': 'MARKET', 'm': True}}, {'str

In [5]:
ws_type = ws_types[2]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(miniTicker) 연결 시도.
  🔗 Websocket Stream(miniTicker) 연결 완료.
  📩 message 수신 시작
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(miniTicker) 연결 종료.
[{'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742385494946, 's': 'BTCUSDT', 'c': '83850.30', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176587.616', 'q': '14543404388.34'}}, {'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742385496777, 's': 'BTCUSDT', 'c': '83857.20', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176588.478', 'q': '14543476668.44'}}, {'stream': 'btcusdt@miniTicker', 'data': {'e': '24hrMiniTicker', 'E': 1742385498827, 's': 'BTCUSDT', 'c': '83868.10', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176589.717', 'q': '14543580575.72'}}, {'stream': 'btcusdt@miniTicker', 'data': {'e': '24h

In [6]:
ws_type = ws_types[3]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(depth) 연결 시도.
  🔗 Websocket Stream(depth) 연결 완료.
  📩 message 수신 시작
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(depth) 연결 종료.
[{'stream': 'btcusdt@depth', 'data': {'e': 'depthUpdate', 'E': 1742385513595, 'T': 1742385513594, 's': 'BTCUSDT', 'U': 7063206102046, 'u': 7063206123071, 'pu': 7063206101981, 'b': [['1000.00', '129.051'], ['5000.00', '4.475'], ['33000.00', '9.960'], ['67095.80', '0.000'], ['79676.30', '0.000'], ['81311.70', '0.000'], ['82381.00', '0.017'], ['82703.90', '1.086'], ['83031.00', '0.079'], ['83035.80', '0.031'], ['83238.60', '0.040'], ['83243.40', '0.008'], ['83352.60', '0.000'], ['83352.70', '1.136'], ['83369.70', '0.035'], ['83446.70', '0.022'], ['83451.50', '0.004'], ['83492.10', '0.000'], ['83495.80', '0.002'], ['83534.20', '0.040'], ['83584.00', '0.002'], ['83584.10', '2.545'], ['83625.20', '0.002

In [7]:
ws_type = ws_types[4]
receiver_count = 10
data = asyncio.run(ws_cycle(ws_type, receiver_count))
storage_message.set_data(ws_type, data)
print(storage_message.get_data(ws_type))

  ⏳ Websocket Stream(aggTrade) 연결 시도.
  🔗 Websocket Stream(aggTrade) 연결 완료.
  📩 message 수신 시작
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
  📩 message 수신 완료
  ⛓️‍💥 Websocket Stream(aggTrade) 연결 종료.
[{'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742385516441, 'a': 2639759222, 's': 'BTCUSDT', 'p': '83869.80', 'q': '0.032', 'f': 6117924375, 'l': 6117924375, 'T': 1742385516286, 'm': False}}, {'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742385516639, 'a': 2639759223, 's': 'BTCUSDT', 'p': '83869.70', 'q': '0.002', 'f': 6117924376, 'l': 6117924376, 'T': 1742385516586, 'm': True}}, {'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 1742385516790, 'a': 2639759224, 's': 'BTCUSDT', 'p': '83869.80', 'q': '0.010', 'f': 6117924377, 'l': 6117924378, 'T': 1742385516635, 'm': False}}, {'stream': 'btcusdt@aggTrade', 'data': {'e': 'aggTrade', 'E': 17423855

In [9]:
data = [{'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385472907, 's': 'BTCUSDT', 'p': '1305.90', 'P': '1.582', 'w': '82357.76', 'c': '83846.20', 'Q': '0.047', 'o': '82540.30', 'h': '84088.20', 'l': '81113.50', 'v': '176627.893', 'q': '14546677916.42', 'O': 1742299020000, 'C': 1742385472903, 'F': 6114867777, 'L': 6117923264, 'n': 3054887}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385474903, 's': 'BTCUSDT', 'p': '1313.30', 'P': '1.591', 'w': '82357.79', 'c': '83853.60', 'Q': '0.004', 'o': '82540.30', 'h': '84088.20', 'l': '81113.50', 'v': '176630.836', 'q': '14546924687.44', 'O': 1742299020000, 'C': 1742385474899, 'F': 6114867777, 'L': 6117923329, 'n': 3054952}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385476967, 's': 'BTCUSDT', 'p': '1313.30', 'P': '1.591', 'w': '82357.80', 'c': '83853.60', 'Q': '0.004', 'o': '82540.30', 'h': '84088.20', 'l': '81113.50', 'v': '176632.130', 'q': '14547033193.95', 'O': 1742299020000, 'C': 1742385476963, 'F': 6114867777, 'L': 6117923358, 'n': 3054981}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385478882, 's': 'BTCUSDT', 'p': '1305.90', 'P': '1.582', 'w': '82357.83', 'c': '83846.20', 'Q': '0.002', 'o': '82540.30', 'h': '84088.20', 'l': '81113.50', 'v': '176636.162', 'q': '14547371282.35', 'O': 1742299020000, 'C': 1742385478879, 'F': 6114867777, 'L': 6117923447, 'n': 3055070}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385480942, 's': 'BTCUSDT', 'p': '1265.70', 'P': '1.533', 'w': '82357.80', 'c': '83840.10', 'Q': '0.100', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176562.857', 'q': '14541328483.38', 'O': 1742299080000, 'C': 1742385480939, 'F': 6114869413, 'L': 6117923512, 'n': 3053499}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385482936, 's': 'BTCUSDT', 'p': '1265.70', 'P': '1.533', 'w': '82357.83', 'c': '83840.10', 'Q': '0.004', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176566.112', 'q': '14541601382.59', 'O': 1742299080000, 'C': 1742385482931, 'F': 6114869413, 'L': 6117923525, 'n': 3053512}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385484862, 's': 'BTCUSDT', 'p': '1259.90', 'P': '1.526', 'w': '82357.88', 'c': '83834.30', 'Q': '0.007', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176572.038', 'q': '14542098215.99', 'O': 1742299080000, 'C': 1742385484859, 'F': 6114869413, 'L': 6117923605, 'n': 3053592}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385486970, 's': 'BTCUSDT', 'p': '1275.30', 'P': '1.544', 'w': '82357.90', 'c': '83849.70', 'Q': '0.101', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176575.117', 'q': '14542356354.37', 'O': 1742299080000, 'C': 1742385486966, 'F': 6114869413, 'L': 6117923698, 'n': 3053685}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385488957, 's': 'BTCUSDT', 'p': '1275.20', 'P': '1.544', 'w': '82357.91', 'c': '83849.60', 'Q': '0.047', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176575.372', 'q': '14542377736.03', 'O': 1742299080000, 'C': 1742385488954, 'F': 6114869413, 'L': 6117923705, 'n': 3053692}}, {'stream': 'btcusdt@ticker', 'data': {'e': '24hrTicker', 'E': 1742385490962, 's': 'BTCUSDT', 'p': '1275.30', 'P': '1.544', 'w': '82357.91', 'c': '83849.70', 'Q': '0.092', 'o': '82574.40', 'h': '84088.20', 'l': '81113.50', 'v': '176575.763', 'q': '14542410521.24', 'O': 1742299080000, 'C': 1742385490957, 'F': 6114869413, 'L': 6117923726, 'n': 3053713}}]

In [10]:
data[0]

{'stream': 'btcusdt@ticker',
 'data': {'e': '24hrTicker',
  'E': 1742385472907,
  's': 'BTCUSDT',
  'p': '1305.90',
  'P': '1.582',
  'w': '82357.76',
  'c': '83846.20',
  'Q': '0.047',
  'o': '82540.30',
  'h': '84088.20',
  'l': '81113.50',
  'v': '176627.893',
  'q': '14546677916.42',
  'O': 1742299020000,
  'C': 1742385472903,
  'F': 6114867777,
  'L': 6117923264,
  'n': 3054887}}

In [11]:
data = [{'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385491584, 'T': 1742385491584, 's': 'BTCUSDT', 't': 6117923730, 'p': '83849.70', 'q': '0.013', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385491657, 'T': 1742385491657, 's': 'BTCUSDT', 't': 6117923731, 'p': '83849.60', 'q': '0.050', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492047, 'T': 1742385492047, 's': 'BTCUSDT', 't': 6117923732, 'p': '83849.60', 'q': '0.514', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492063, 'T': 1742385492062, 's': 'BTCUSDT', 't': 6117923733, 'p': '83849.60', 'q': '0.001', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492219, 'T': 1742385492219, 's': 'BTCUSDT', 't': 6117923734, 'p': '83849.70', 'q': '0.051', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492398, 'T': 1742385492398, 's': 'BTCUSDT', 't': 6117923735, 'p': '83849.60', 'q': '0.014', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492617, 'T': 1742385492617, 's': 'BTCUSDT', 't': 6117923736, 'p': '83849.70', 'q': '0.014', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492624, 'T': 1742385492623, 's': 'BTCUSDT', 't': 6117923737, 'p': '83849.70', 'q': '0.004', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492639, 'T': 1742385492639, 's': 'BTCUSDT', 't': 6117923738, 'p': '83849.70', 'q': '0.002', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492904, 'T': 1742385492904, 's': 'BTCUSDT', 't': 6117923739, 'p': '83849.70', 'q': '0.011', 'X': 'MARKET', 'm': False}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385492918, 'T': 1742385492918, 's': 'BTCUSDT', 't': 6117923740, 'p': '83849.60', 'q': '0.023', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493201, 'T': 1742385493201, 's': 'BTCUSDT', 't': 6117923741, 'p': '83849.60', 'q': '0.002', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493421, 'T': 1742385493421, 's': 'BTCUSDT', 't': 6117923742, 'p': '83849.60', 'q': '0.004', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493477, 'T': 1742385493477, 's': 'BTCUSDT', 't': 6117923743, 'p': '83849.60', 'q': '0.006', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493477, 'T': 1742385493477, 's': 'BTCUSDT', 't': 6117923744, 'p': '83849.60', 'q': '0.002', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493477, 'T': 1742385493477, 's': 'BTCUSDT', 't': 6117923745, 'p': '83849.60', 'q': '0.010', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493477, 'T': 1742385493477, 's': 'BTCUSDT', 't': 6117923746, 'p': '83849.60', 'q': '0.013', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493619, 'T': 1742385493619, 's': 'BTCUSDT', 't': 6117923747, 'p': '83849.60', 'q': '0.014', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493619, 'T': 1742385493619, 's': 'BTCUSDT', 't': 6117923748, 'p': '83849.60', 'q': '0.003', 'X': 'MARKET', 'm': True}}, {'stream': 'btcusdt@trade', 'data': {'e': 'trade', 'E': 1742385493619, 'T': 1742385493619, 's': 'BTCUSDT', 't': 6117923749, 'p': '83849.60', 'q': '0.003', 'X': 'MARKET', 'm': True}}]

In [12]:
data[0]


{'stream': 'btcusdt@trade',
 'data': {'e': 'trade',
  'E': 1742385491584,
  'T': 1742385491584,
  's': 'BTCUSDT',
  't': 6117923730,
  'p': '83849.70',
  'q': '0.013',
  'X': 'MARKET',
  'm': False}}

In [13]:
for i in data:
    print(i["data"])

{'e': 'trade', 'E': 1742385491584, 'T': 1742385491584, 's': 'BTCUSDT', 't': 6117923730, 'p': '83849.70', 'q': '0.013', 'X': 'MARKET', 'm': False}
{'e': 'trade', 'E': 1742385491657, 'T': 1742385491657, 's': 'BTCUSDT', 't': 6117923731, 'p': '83849.60', 'q': '0.050', 'X': 'MARKET', 'm': True}
{'e': 'trade', 'E': 1742385492047, 'T': 1742385492047, 's': 'BTCUSDT', 't': 6117923732, 'p': '83849.60', 'q': '0.514', 'X': 'MARKET', 'm': True}
{'e': 'trade', 'E': 1742385492063, 'T': 1742385492062, 's': 'BTCUSDT', 't': 6117923733, 'p': '83849.60', 'q': '0.001', 'X': 'MARKET', 'm': True}
{'e': 'trade', 'E': 1742385492219, 'T': 1742385492219, 's': 'BTCUSDT', 't': 6117923734, 'p': '83849.70', 'q': '0.051', 'X': 'MARKET', 'm': False}
{'e': 'trade', 'E': 1742385492398, 'T': 1742385492398, 's': 'BTCUSDT', 't': 6117923735, 'p': '83849.60', 'q': '0.014', 'X': 'MARKET', 'm': True}
{'e': 'trade', 'E': 1742385492617, 'T': 1742385492617, 's': 'BTCUSDT', 't': 6117923736, 'p': '83849.70', 'q': '0.014', 'X': 'MAR

In [26]:
result = {"buy":0, "sell":0}
for i in data:
    get_data = i["data"]
    value = float(get_data["p"]) * float(get_data["q"])
    if get_data["m"]:
        result["sell"] += value
    else:
        result["buy"] += value
cals = (result["buy"] / result["sell"]) * 100
print(f" {cals:,.2f} %")

 14.42 %


In [20]:
result

{'buy': 6, 'sell': 14}